<a href="https://colab.research.google.com/github/lilyzhizhou/datathon-3/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SyntaxError: ignored